In [1]:
import requests
import pandas as pd 
import numpy as np
import re
import sys, getopt
import csv
import os
import matplotlib.pyplot as plt
import seaborn as sns

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

In [7]:
os.chdir('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings')
os.getcwd()

'/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings'

In [16]:
latest_match_data = load_csv_dataset('create_dataset_1/CSVs we use often/newest_edited_match_dataset_may_30.csv')
pd.set_option('display.max_columns', None)

## Quick dataset cleaning

In [40]:
latest_match_data.loc[latest_match_data['Team.1.Name'] == 'Seychelles', 'Team.1.Code'] = 'SCH'
latest_match_data.loc[latest_match_data['Team.2.Name'] == 'Seychelles', 'Team.2.Code'] = 'SCH'

latest_match_data.loc[latest_match_data['Team.1.Name'] == 'Nicaragua', 'Team.1.Code'] = 'NIC'
latest_match_data.loc[latest_match_data['Team.2.Name'] == 'Nicaragua', 'Team.2.Code'] = 'NIC'

latest_match_data.loc[latest_match_data['Team.1.Name'] == 'Democratic Republic of the Congo', 'Team.1.Name'] = 'DR Congo'
latest_match_data.loc[latest_match_data['Team.2.Name'] == 'Democratic Republic of the Congo', 'Team.2.Name'] = 'DR Congo'

latest_match_data.loc[latest_match_data['Team.1.Name'] == 'Swaziland', 'Team.1.Name'] = 'Eswatini'
latest_match_data.loc[latest_match_data['Team.2.Name'] == 'Swaziland', 'Team.2.Name'] = 'Eswatini'

In [45]:
len(latest_match_data['Team.1.Code'].unique()), len(latest_match_data['Team.1.Name'].unique())

# Group by 'Team.1.Code' and filter groups having more than one unique 'Team.1.Name'
grouped = latest_match_data.groupby('Team.1.Code').filter(lambda x: x['Team.1.Name'].nunique() > 1)

# Display the rows where 'Team.1.Code' has multiple 'Team.1.Name'
print(grouped[['Team.1.Code', 'Team.1.Name']].drop_duplicates())

Empty DataFrame
Columns: [Team.1.Code, Team.1.Name]
Index: []


In [46]:
len(latest_match_data['Team.2.Code'].unique()), len(latest_match_data['Team.2.Name'].unique())

# Group by 'Team.1.Code' and filter groups having more than one unique 'Team.1.Name'
grouped = latest_match_data.groupby('Team.2.Code').filter(lambda x: x['Team.2.Name'].nunique() > 1)

# Display the rows where 'Team.1.Code' has multiple 'Team.1.Name'
print(grouped[['Team.2.Code', 'Team.2.Name']].drop_duplicates())

Empty DataFrame
Columns: [Team.2.Code, Team.2.Name]
Index: []


## Task 1: does each team's win rate get higher the larger their MV difference

### Script Version

In [250]:
# Assuming latest_match_data is your DataFrame
team1_unique = latest_match_data['Team.1.Name'].unique()
team2_unique = latest_match_data['Team.2.Name'].unique()

# Create the union of both unique lists
all_teams_unique = np.union1d(team1_unique, team2_unique)

In [287]:
overall_table = pd.DataFrame()
for team in all_teams_unique:
    this_team_table = generate_win_rate_intervals_table_team(team, latest_match_data)
    overall_table = pd.concat([overall_table, this_team_table], ignore_index=True)


/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_53520/1093373292.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_table = pd.concat([result_table, pd.DataFrame([new_row])], ignore_index=True)
/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_53520/1093373292.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  more_valuable_matches['Value Difference (Abs)'] = abs(result_table['Market Value Difference'])
/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_53520/109

In [297]:
overall_table.to_csv('overall_table.csv', index=False)

In [279]:
def generate_win_rate_intervals_table_team(first_team, latest_match_data):

    #####PAGE 1#####

    # Filter the rows where the team is either in 'Team.1.Name' or 'Team.2.Name'
    team_data = latest_match_data[(latest_match_data['Team.1.Name'] == first_team) | (latest_match_data['Team.2.Name'] == first_team)]

    # Create a new DataFrame to store the results
    columns = ['Country Name', 'Date', 'Competition', 'Score Difference', 'Market Value Difference', 'Team.1 Total Market Value', 'Team.2 Total Market Value', 'Season', 'Team Position']
    result_table = pd.DataFrame(columns=columns)

    # Populate the DataFrame
    for index, row in team_data.iterrows():
        if row['Team.1.Name'] == first_team:
            new_row = {
                'Country Name': first_team,
                'Date': row['Date'],
                'Competition': row['Competition'],
                'Score Difference': row['Score.Difference'],
                'Market Value Difference': row['Market.Value.Difference'],
                'Team.1 Total Market Value': row['Team.1.Total.Market.Value'],
                'Team.2 Total Market Value': row['Team.2.Total.Market.Value'],
                'Season': row['Season'],
                'Team Position': 'Team 1'
            }
        else:
            new_row = {
                'Country Name': first_team,
                'Date': row['Date'],
                'Competition': row['Competition'],
                'Score Difference': row['Score.Difference'],
                'Market Value Difference': row['Market.Value.Difference'],
                'Team.1 Total Market Value': row['Team.1.Total.Market.Value'],
                'Team.2 Total Market Value': row['Team.2.Total.Market.Value'],
                'Season': row['Season'],
                'Team Position': 'Team 2'
            }
        result_table = pd.concat([result_table, pd.DataFrame([new_row])], ignore_index=True)

    #####PAGE 2#####
    # Add new columns with default values
    result_table['Team Won?'] = False
    result_table['Team More Valuable?'] = False

    # Update the columns based on conditions
    for index, row in result_table.iterrows():
        if row['Team Position'] == 'Team 1':
            if row['Score Difference'] > 0:
                result_table.at[index, 'Team Won?'] = True
            if row['Market Value Difference'] > 0:
                result_table.at[index, 'Team More Valuable?'] = True
        elif row['Team Position'] == 'Team 2':
            if row['Score Difference'] < 0:
                result_table.at[index, 'Team Won?'] = True
            if row['Market Value Difference'] < 0:
                result_table.at[index, 'Team More Valuable?'] = True

    #####PAGE 3#####

    # Calculate the total number of matches
    less_valuable_matches = result_table[result_table['Team More Valuable?'] == False]
    more_valuable_matches = result_table[result_table['Team More Valuable?'] == True]
    if len(more_valuable_matches) > 0:
        more_valuable_matches['Value Difference (Abs)'] = abs(result_table['Market Value Difference'])
    else:
        0==0
    if len(less_valuable_matches) > 0:
        less_valuable_matches['Value Difference (Abs)'] = abs(result_table['Market Value Difference']) * -1


    #####PAGE 4#####
    #PRODUCING TABLE FROM MORE VALUABLE MATCHES
    interval_stats_ABOVE = pd.DataFrame(columns=['Team Name', 'Interval Range', 'Total Matches', 'Total Wins', 'Win Percentage'])
    if len(more_valuable_matches) == 0:
        0==0
    else:
        for i in range(5, 9):
            if (i == 7) or (i == 8) or (i == 6):
                number_lower = 10 ** i
                number_higher = 10 ** (i + 1)
                interval_matches = more_valuable_matches[(more_valuable_matches['Value Difference (Abs)'] >= number_lower) & (more_valuable_matches['Value Difference (Abs)'] <= number_higher / 2)]
                num_matches = len(interval_matches)
                interval_wins = interval_matches[interval_matches['Team Won?'] == True]
                num_wins = len(interval_wins)
                win_percentage = (num_wins / num_matches) * 100 if num_matches > 0 else 0
                interval_row = {
                    'Team Name': first_team,
                    'Interval Range': f"${number_lower} - ${number_higher / 2}",
                    'Total Matches': num_matches,
                    'Total Wins': num_wins,
                    'Win Percentage': win_percentage
                }
                interval_stats_ABOVE = pd.concat([interval_stats_ABOVE, pd.DataFrame([interval_row])], ignore_index=True)

                interval_matches_ii = more_valuable_matches[(more_valuable_matches['Value Difference (Abs)'] >= (number_higher / 2)) & (more_valuable_matches['Value Difference (Abs)'] <= number_higher)]
                num_matches_ii = len(interval_matches_ii)
                interval_wins_ii = interval_matches_ii[interval_matches_ii['Team Won?'] == True]
                num_wins_ii = len(interval_wins_ii)
                win_percentage_ii = (num_wins_ii / num_matches_ii) * 100 if num_matches_ii > 0 else 0
                interval_row = {
                    'Team Name': first_team,
                    'Interval Range': f"${number_higher / 2} - ${number_higher}",
                    'Total Matches': num_matches_ii,
                    'Total Wins': num_wins_ii,
                    'Win Percentage': win_percentage_ii
                }
                interval_stats_ABOVE = pd.concat([interval_stats_ABOVE, pd.DataFrame([interval_row])], ignore_index=True)

            else:
                number_lower = 10 ** i
                number_higher = 10 ** (i + 1)
                interval_matches = more_valuable_matches[(more_valuable_matches['Value Difference (Abs)'] >= number_lower) & (more_valuable_matches['Value Difference (Abs)'] <= number_higher)]
                num_matches = len(interval_matches)
                interval_wins = interval_matches[interval_matches['Team Won?'] == True]
                num_wins = len(interval_wins)
                win_percentage = (num_wins / num_matches) * 100 if num_matches > 0 else 0
                interval_row = {
                    'Team Name': first_team,
                    'Interval Range': f"${number_lower} - ${number_higher / 2}",
                    'Total Matches': num_matches,
                    'Total Wins': num_wins,
                    'Win Percentage': win_percentage
                }
                interval_stats_ABOVE = pd.concat([interval_stats_ABOVE, pd.DataFrame([interval_row])], ignore_index=True)
        interval_stats_ABOVE['Interval Side'] = 'More Valuable'

    #####PAGE 5#####
    #PRODUCING TABLE FROM LESS VALUABLE MATCHES
    interval_stats_BELOW = pd.DataFrame(columns=['Team Name', 'Interval Range', 'Total Matches', 'Total Wins', 'Win Percentage'])
    if len(less_valuable_matches) == 0:
        0==0
    else:
        for i in range(5, 9):
            if (i == 7) or (i == 8) or (i == 6):
                number_lower = (10 ** i) * -1
                number_higher = (10 ** (i + 1)) * -1
                interval_matches = less_valuable_matches[(less_valuable_matches['Value Difference (Abs)'] <= number_lower) & (less_valuable_matches['Value Difference (Abs)'] >= number_higher / 2)]
                num_matches = len(interval_matches)
                interval_wins = interval_matches[interval_matches['Team Won?'] == True]
                num_wins = len(interval_wins)
                win_percentage = (num_wins / num_matches) * 100 if num_matches > 0 else 0
                interval_row = {
                    'Team Name': first_team,
                    'Interval Range': f"${number_lower} - ${number_higher / 2}",
                    'Total Matches': num_matches,
                    'Total Wins': num_wins,
                    'Win Percentage': win_percentage
                }
                interval_stats_BELOW = pd.concat([interval_stats_BELOW, pd.DataFrame([interval_row])], ignore_index=True)

                interval_matches_ii = less_valuable_matches[(less_valuable_matches['Value Difference (Abs)'] <= (number_higher / 2)) & (less_valuable_matches['Value Difference (Abs)'] >= number_higher)]
                num_matches_ii = len(interval_matches_ii)
                interval_wins_ii = interval_matches_ii[interval_matches_ii['Team Won?'] == True]
                num_wins_ii = len(interval_wins_ii)
                win_percentage_ii = (num_wins_ii / num_matches_ii) * 100 if num_matches_ii > 0 else 0
                interval_row = {
                    'Team Name': first_team,
                    'Interval Range': f"${number_higher / 2} - ${number_higher}",
                    'Total Matches': num_matches_ii,
                    'Total Wins': num_wins_ii,
                    'Win Percentage': win_percentage_ii
                }
                interval_stats_BELOW = pd.concat([interval_stats_BELOW, pd.DataFrame([interval_row])], ignore_index=True)

            else:
                number_lower = (10 ** i) * -1
                number_higher = (10 ** (i + 1)) * -1
                interval_matches = less_valuable_matches[(less_valuable_matches['Value Difference (Abs)'] <= number_lower) & (less_valuable_matches['Value Difference (Abs)'] >= number_higher)]
                num_matches = len(interval_matches)
                interval_wins = interval_matches[interval_matches['Team Won?'] == True]
                num_wins = len(interval_wins)
                win_percentage = (num_wins / num_matches) * 100 if num_matches > 0 else 0
                interval_row = {
                    'Team Name': first_team,
                    'Interval Range': f"${number_lower} - ${number_higher / 2}",
                    'Total Matches': num_matches,
                    'Total Wins': num_wins,
                    'Win Percentage': win_percentage
                }
                interval_stats_BELOW = pd.concat([interval_stats_BELOW, pd.DataFrame([interval_row])], ignore_index=True)
        interval_stats_BELOW['Interval Side'] = 'Less Valuable'

    interval_stats_final = pd.concat([interval_stats_ABOVE, interval_stats_BELOW], ignore_index=True)
    return interval_stats_final

### Creating Script (OLD)

In [266]:
#####PAGE 1#####


#for each Unique Team Name in latest_match_data


#if the Team Name is Team.1.Name, add that Score.Difference and market Market.Value.Difference as a row to the table
    #add the opposing team name (Team.2.Name), add Team.1.Total.Market.Value, add Team.2.Total.Market.Value	
    #add the Date and Competition and Season

#if the Team Name is Team.2.Name, add (-1* that Score.Difference) and (-1 * market Market.Value.Difference) as a row to the table
    #add the opposing team name (Team.2.Name), add Team.1.Total.Market.Value, add Team.2.Total.Market.Value	
    #add the Date and Competition and Season

#you want a table with 
    #Country name (all rows will be the same)
    #Date 
    #Competition
    #Score.Difference
    #Market.Value.Difference
    #Team.1.Total.Market.Value, add Team.2.Total.Market.Value	
    #Season

#give me the code to do this for just latest_match_data['Team.1.Name'].unique()[0], the first team in the unique list of teams. I will turn it into a loop later. 

# Extract the first unique team name from 'Team.1.Name'
first_team = 'England' #latest_match_data['Team.1.Name'].unique()[0]

# Filter the rows where the team is either in 'Team.1.Name' or 'Team.2.Name'
team_data = latest_match_data[(latest_match_data['Team.1.Name'] == first_team) | (latest_match_data['Team.2.Name'] == first_team)]

# Create a new DataFrame to store the results
columns = ['Country Name', 'Date', 'Competition', 'Score Difference', 'Market Value Difference', 'Team.1 Total Market Value', 'Team.2 Total Market Value', 'Season', 'Team Position']
result_table = pd.DataFrame(columns=columns)

# Populate the DataFrame
for index, row in team_data.iterrows():
    if row['Team.1.Name'] == first_team:
        new_row = {
            'Country Name': first_team,
            'Date': row['Date'],
            'Competition': row['Competition'],
            'Score Difference': row['Score.Difference'],
            'Market Value Difference': row['Market.Value.Difference'],
            'Team.1 Total Market Value': row['Team.1.Total.Market.Value'],
            'Team.2 Total Market Value': row['Team.2.Total.Market.Value'],
            'Season': row['Season'],
            'Team Position': 'Team 1'
        }
    else:
        new_row = {
            'Country Name': first_team,
            'Date': row['Date'],
            'Competition': row['Competition'],
            'Score Difference': row['Score.Difference'],
            'Market Value Difference': row['Market.Value.Difference'],
            'Team.1 Total Market Value': row['Team.1.Total.Market.Value'],
            'Team.2 Total Market Value': row['Team.2.Total.Market.Value'],
            'Season': row['Season'],
            'Team Position': 'Team 2'
        }
    new_df = pd.DataFrame([new_row])
    new_df = new_df.dropna(axis=1, how='all')

    result_table = pd.concat([result_table, new_df], ignore_index=True)

result_table


/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_53520/2190568479.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_table = pd.concat([result_table, new_df], ignore_index=True)


,Country Name,Date,Competition,Score Difference,Market Value Difference,Team.1 Total Market Value,Team.2 Total Market Value,Season,Team Position
0,England,2014-09-08,"Euros, Qualifiers",-2,-110500000.0,116500000.0,227000000.0,2014,Team 2
1,England,2014-10-09,"Euros, Qualifiers",5,221375000.0,221500000.0,125000.0,2014,Team 1
2,England,2014-10-12,"Euros, Qualifiers",-1,-205300000.0,6200000.0,211500000.0,2014,Team 2
3,England,2014-11-15,"Euros, Qualifiers",2,188950000.0,229500000.0,40550000.0,2014,Team 1
4,England,2015-03-27,"Euros, Qualifiers",4,186592500.0,193600000.0,7007500.0,2015,Team 1
...,...,...,...,...,...,...,...,...,...
93,England,2021-09-08,"World Cup Qualifiers, Europe",0,-402500000.0,157500000.0,560000000.0,2021,Team 2
94,England,2021-10-09,"World Cup Qualifiers, Europe",-5,-453980000.0,920000.0,454900000.0,2021,Team 2
95,England,2021-10-12,"World Cup Qualifiers, Europe",0,531850000.0,580000000.0,48150000.0,2021,Team 1
96,England,2021-11-12,"World Cup Qualifiers, Europe",5,495085000.0,573500000.0,78415000.0,2021,Team 1


In [267]:
#####PAGE 2#####


# Add new columns with default values
result_table['Team Won?'] = False
result_table['Team More Valuable?'] = False

# Update the columns based on conditions
for index, row in result_table.iterrows():
    if row['Team Position'] == 'Team 1':
        if row['Score Difference'] > 0:
            result_table.at[index, 'Team Won?'] = True
        if row['Market Value Difference'] > 0:
            result_table.at[index, 'Team More Valuable?'] = True
    elif row['Team Position'] == 'Team 2':
        if row['Score Difference'] < 0:
            result_table.at[index, 'Team Won?'] = True
        if row['Market Value Difference'] < 0:
            result_table.at[index, 'Team More Valuable?'] = True

result_table

,Country Name,Date,Competition,Score Difference,Market Value Difference,Team.1 Total Market Value,Team.2 Total Market Value,Season,Team Position,Team Won?,Team More Valuable?
0,England,2014-09-08,"Euros, Qualifiers",-2,-110500000.0,116500000.0,227000000.0,2014,Team 2,True,True
1,England,2014-10-09,"Euros, Qualifiers",5,221375000.0,221500000.0,125000.0,2014,Team 1,True,True
2,England,2014-10-12,"Euros, Qualifiers",-1,-205300000.0,6200000.0,211500000.0,2014,Team 2,True,True
3,England,2014-11-15,"Euros, Qualifiers",2,188950000.0,229500000.0,40550000.0,2014,Team 1,True,True
4,England,2015-03-27,"Euros, Qualifiers",4,186592500.0,193600000.0,7007500.0,2015,Team 1,True,True
...,...,...,...,...,...,...,...,...,...,...,...
93,England,2021-09-08,"World Cup Qualifiers, Europe",0,-402500000.0,157500000.0,560000000.0,2021,Team 2,False,True
94,England,2021-10-09,"World Cup Qualifiers, Europe",-5,-453980000.0,920000.0,454900000.0,2021,Team 2,True,True
95,England,2021-10-12,"World Cup Qualifiers, Europe",0,531850000.0,580000000.0,48150000.0,2021,Team 1,False,True
96,England,2021-11-12,"World Cup Qualifiers, Europe",5,495085000.0,573500000.0,78415000.0,2021,Team 1,True,True


In [271]:
#####PAGE 3#####

# Calculate the total number of matches
less_valuable_matches = result_table[result_table['Team More Valuable?'] == False]
more_valuable_matches = result_table[result_table['Team More Valuable?'] == True]
total_num_matches_more_valuable = len(more_valuable_matches)
if len(more_valuable_matches) > 0:
    more_valuable_matches['Value Difference (Abs)'] = abs(result_table['Market Value Difference'])
else:
    0==0
total_num_matches_less_valuable = len(less_valuable_matches)
if len(less_valuable_matches) > 0:
    less_valuable_matches['Value Difference (Abs)'] = abs(result_table['Market Value Difference']) * -1

In [273]:
#####PAGE 4#####

#PRODUCING TABLE FROM MORE VALUABLE MATCHES

# Create a DataFrame to store the results
interval_stats = pd.DataFrame(columns=['Team Name', 'Interval Range', 'Total Matches', 'Total Wins', 'Win Percentage'])
for i in range(5, 9):
    if (i == 7) or (i == 8) or (i == 6):
        number_lower = 10 ** i
        number_higher = 10 ** (i + 1)
        interval_matches = more_valuable_matches[(more_valuable_matches['Value Difference (Abs)'] >= number_lower) & (more_valuable_matches['Value Difference (Abs)'] <= number_higher / 2)]
        num_matches = len(interval_matches)
        interval_wins = interval_matches[interval_matches['Team Won?'] == True]
        num_wins = len(interval_wins)
        win_percentage = (num_wins / num_matches) * 100 if num_matches > 0 else 0
        interval_row = {
            'Team Name': first_team,
            'Interval Range': f"${number_lower} - ${number_higher / 2}",
            'Total Matches': num_matches,
            'Total Wins': num_wins,
            'Win Percentage': win_percentage
        }
        interval_stats = pd.concat([interval_stats, pd.DataFrame([interval_row])], ignore_index=True)

        interval_matches_ii = more_valuable_matches[(more_valuable_matches['Value Difference (Abs)'] >= (number_higher / 2)) & (more_valuable_matches['Value Difference (Abs)'] <= number_higher)]
        num_matches_ii = len(interval_matches_ii)
        interval_wins_ii = interval_matches_ii[interval_matches_ii['Team Won?'] == True]
        num_wins_ii = len(interval_wins_ii)
        win_percentage_ii = (num_wins_ii / num_matches_ii) * 100 if num_matches_ii > 0 else 0
        interval_row = {
            'Team Name': first_team,
            'Interval Range': f"${number_higher / 2} - ${number_higher}",
            'Total Matches': num_matches_ii,
            'Total Wins': num_wins_ii,
            'Win Percentage': win_percentage_ii
        }
        interval_stats = pd.concat([interval_stats, pd.DataFrame([interval_row])], ignore_index=True)

    else:
        number_lower = 10 ** i
        number_higher = 10 ** (i + 1)
        interval_matches = more_valuable_matches[(more_valuable_matches['Value Difference (Abs)'] >= number_lower) & (more_valuable_matches['Value Difference (Abs)'] <= number_higher)]
        num_matches = len(interval_matches)
        interval_wins = interval_matches[interval_matches['Team Won?'] == True]
        num_wins = len(interval_wins)
        win_percentage = (num_wins / num_matches) * 100 if num_matches > 0 else 0
        interval_row = {
            'Team Name': first_team,
            'Interval Range': f"${number_lower} - ${number_higher / 2}",
            'Total Matches': num_matches,
            'Total Wins': num_wins,
            'Win Percentage': win_percentage
        }
        interval_stats = pd.concat([interval_stats, pd.DataFrame([interval_row])], ignore_index=True)
interval_stats['Interval Side'] = 'More Valuable'
interval_stats

,Team Name,Interval Range,Total Matches,Total Wins,Win Percentage,Interval Side
0,England,$100000 - $500000.0,0,0,0,More Valuable
1,England,$1000000 - $5000000.0,0,0,0,More Valuable
2,England,$5000000.0 - $10000000,1,0,0.0,More Valuable
3,England,$10000000 - $50000000.0,2,1,50.0,More Valuable
4,England,$50000000.0 - $100000000,9,2,22.222222,More Valuable
5,England,$100000000 - $500000000.0,58,37,63.793103,More Valuable
6,England,$500000000.0 - $1000000000,28,23,82.142857,More Valuable


In [277]:
#####PAGE 5#####

#PRODUCING TABLE FROM LESS VALUABLE MATCHES

if len(less_valuable_matches) == 0:
    interval_stats = pd.DataFrame(columns=['Team Name', 'Interval Range', 'Total Matches', 'Total Wins', 'Win Percentage'])
else:
    0==0

    # Create a DataFrame to store the results
    interval_stats = pd.DataFrame(columns=['Team Name', 'Interval Range', 'Total Matches', 'Total Wins', 'Win Percentage'])
    for i in range(5, 9):
        if (i == 7) or (i == 8) or (i == 6):
            number_lower = (10 ** i) * -1
            number_higher = (10 ** (i + 1)) * -1
            interval_matches = less_valuable_matches[(less_valuable_matches['Value Difference (Abs)'] <= number_lower) & (less_valuable_matches['Value Difference (Abs)'] >= number_higher / 2)]
            num_matches = len(interval_matches)
            interval_wins = interval_matches[interval_matches['Team Won?'] == True]
            num_wins = len(interval_wins)
            win_percentage = (num_wins / num_matches) * 100 if num_matches > 0 else 0
            interval_row = {
                'Team Name': first_team,
                'Interval Range': f"${number_lower} - ${number_higher / 2}",
                'Total Matches': num_matches,
                'Total Wins': num_wins,
                'Win Percentage': win_percentage
            }
            interval_stats = pd.concat([interval_stats, pd.DataFrame([interval_row])], ignore_index=True)

            interval_matches_ii = less_valuable_matches[(less_valuable_matches['Value Difference (Abs)'] <= (number_higher / 2)) & (less_valuable_matches['Value Difference (Abs)'] >= number_higher)]
            num_matches_ii = len(interval_matches_ii)
            interval_wins_ii = interval_matches_ii[interval_matches_ii['Team Won?'] == True]
            num_wins_ii = len(interval_wins_ii)
            win_percentage_ii = (num_wins_ii / num_matches_ii) * 100 if num_matches_ii > 0 else 0
            interval_row = {
                'Team Name': first_team,
                'Interval Range': f"${number_higher / 2} - ${number_higher}",
                'Total Matches': num_matches_ii,
                'Total Wins': num_wins_ii,
                'Win Percentage': win_percentage_ii
            }
            interval_stats = pd.concat([interval_stats, pd.DataFrame([interval_row])], ignore_index=True)

        else:
            number_lower = (10 ** i) * -1
            number_higher = (10 ** (i + 1)) * -1
            interval_matches = less_valuable_matches[(less_valuable_matches['Value Difference (Abs)'] <= number_lower) & (less_valuable_matches['Value Difference (Abs)'] >= number_higher)]
            num_matches = len(interval_matches)
            interval_wins = interval_matches[interval_matches['Team Won?'] == True]
            num_wins = len(interval_wins)
            win_percentage = (num_wins / num_matches) * 100 if num_matches > 0 else 0
            interval_row = {
                'Team Name': first_team,
                'Interval Range': f"${number_lower} - ${number_higher / 2}",
                'Total Matches': num_matches,
                'Total Wins': num_wins,
                'Win Percentage': win_percentage
            }
            interval_stats = pd.concat([interval_stats, pd.DataFrame([interval_row])], ignore_index=True)
    interval_stats['Interval Side'] = 'Less Valuable'
interval_stats

,Team Name,Interval Range,Total Matches,Total Wins,Win Percentage


In [ ]:
#####NOT A PAGE#####

# Calculate the total number of matches
less_valuable_matches = result_table[result_table['Team More Valuable?'] == False]
more_valuable_matches = result_table[result_table['Team More Valuable?'] == True]
total_num_matches_more_valuable = len(more_valuable_matches)
more_valuable_matches['Value Difference (Abs)'] = abs(result_table['Market Value Difference'])

for i in range(5, 9):
    if (i == 7) or (i == 8) or (i == 6):
        print(i, '**')
        number_lower = 10 ** i
        number_higher = 10 ** (i + 1)
        interval_matches = more_valuable_matches[(more_valuable_matches['Value Difference (Abs)'] >= number_lower) & (more_valuable_matches['Value Difference (Abs)'] <= number_higher / 2)]
        num_matches = len(interval_matches)
        interval_wins = interval_matches[interval_matches['Team Won?'] == True]
        num_wins = len(interval_wins)
        print(number_lower, number_higher / 2, num_matches, num_wins)
        interval_matches_ii = more_valuable_matches[(more_valuable_matches['Value Difference (Abs)'] >= (number_higher / 2)) & (more_valuable_matches['Value Difference (Abs)'] <= number_higher)]
        num_matches_ii = len(interval_matches_ii)
        interval_wins_ii = interval_matches_ii[interval_matches_ii['Team Won?'] == True]
        num_wins_ii = len(interval_wins_ii)
        if num_matches_ii != 0:
            print((number_higher / 2), number_higher, num_matches_ii, num_wins_ii, num_wins_ii / num_matches_ii)
        else:
            print((number_higher / 2), number_higher, num_matches_ii, num_wins_ii, 0)
    else:
        print(i)
        number_lower = 10 ** i
        number_higher = 10 ** (i + 1)
        interval_matches = more_valuable_matches[(more_valuable_matches['Value Difference (Abs)'] >= number_lower) & (more_valuable_matches['Value Difference (Abs)'] <= number_higher)]
        num_matches = len(interval_matches)
        interval_wins = interval_matches[interval_matches['Team Won?'] == True]
        num_wins = len(interval_wins)
        print(number_lower, number_higher, num_matches, num_wins, num_wins / num_matches)
        if num_matches != 0:
            print((number_higher / 2), number_higher, num_matches, num_wins, num_wins/ num_matches)
        else:
            print((number_higher / 2), number_higher, num_matches_ii, num_wins_ii, 0)


## Task 2 - UEFA vs everyone else

In [165]:
# Filter the DataFrame to find matches where one team is from UEFA and the other is not
uefa_vs_others = latest_match_data[
    ((latest_match_data['Team.1.Confederation'] == 'UEFA') & (latest_match_data['Team.2.Confederation'] != 'UEFA')) |
    ((latest_match_data['Team.2.Confederation'] == 'UEFA') & (latest_match_data['Team.1.Confederation'] != 'UEFA'))
]


uefa_vs_uefa = latest_match_data[
    ((latest_match_data['Team.1.Confederation'] == 'UEFA') & (latest_match_data['Team.2.Confederation'] == 'UEFA'))]

In [173]:
uefa_vs_others['Competition'].value_counts()


Competition
World Cup    117
Olympics      30
Name: count, dtype: int64

In [167]:
uefa_vs_uefa['Competition'].value_counts()

Competition
World Cup Qualifiers, Europe    780
Euros, Qualifiers               740
Nations League                  454
Euros                           102
World Cup                        29
Olympics                          1
Name: count, dtype: int64

In [189]:
#for each team in all_teams_uefa
 
#their win PCT when they play less valuable teams that are in UEFA 
#their win PCT when they play more valuable teams that are in UEFA

#their win PCT when they play less valuable teams that are NOT in UEFA 
#their win PCT when they play more valuable teams that are NOT in UEFA

set_team_1 = uefa_vs_others[uefa_vs_others['Team.1.Confederation'] == 'UEFA']['Team.1.Name'].unique()
set_team_2 = uefa_vs_others[uefa_vs_others['Team.2.Confederation'] == 'UEFA']['Team.2.Name'].unique()

all_teams_uefa = np.union1d(set_team_1, set_team_2)

In [209]:
win_pct_stats = pd.DataFrame()
for team in all_teams_uefa:
    # Filter matches involving the team
    team_matches_vs_not_uefa = uefa_vs_others[(uefa_vs_others['Team.1.Name'] == team) | (uefa_vs_others['Team.2.Name'] == team)]
    team_matches_vs_uefa = uefa_vs_uefa[(uefa_vs_uefa['Team.1.Name'] == team) | (uefa_vs_uefa['Team.2.Name'] == team)]


    num_matches_vs_not_uefa = len(team_matches_vs_not_uefa)
    num_matches_vs_uefa = len(team_matches_vs_uefa)


    #PLAYING NON-UEFA CONFEDERATIONS
    team_matches_vs_not_uefa_more_valuable = team_matches_vs_not_uefa[
        ((team_matches_vs_not_uefa['Team.1.Name'] == team) & (team_matches_vs_not_uefa['Team.1.Total.Market.Value'] > team_matches_vs_not_uefa['Team.2.Total.Market.Value'])) |
        ((team_matches_vs_not_uefa['Team.2.Name'] == team) & (team_matches_vs_not_uefa['Team.1.Total.Market.Value'] < team_matches_vs_not_uefa['Team.2.Total.Market.Value']))
    ]
    team_matches_vs_not_uefa_less_valuable = team_matches_vs_not_uefa[
        ((team_matches_vs_not_uefa['Team.1.Name'] == team) & (team_matches_vs_not_uefa['Team.1.Total.Market.Value'] < team_matches_vs_not_uefa['Team.2.Total.Market.Value'])) |
        ((team_matches_vs_not_uefa['Team.2.Name'] == team) & (team_matches_vs_not_uefa['Team.1.Total.Market.Value'] > team_matches_vs_not_uefa['Team.2.Total.Market.Value']))
    ]

    wins_vs_not_uefa_more_valuable = team_matches_vs_not_uefa_more_valuable[
        ((team_matches_vs_not_uefa_more_valuable['Team.1.Name'] == team) & (team_matches_vs_not_uefa_more_valuable['Score.Difference'] > 0)) |
        ((team_matches_vs_not_uefa_more_valuable['Team.2.Name'] == team) & (team_matches_vs_not_uefa_more_valuable['Score.Difference'] < 0))
    ]

    wins_vs_not_uefa_less_valuable = team_matches_vs_not_uefa_less_valuable[
        ((team_matches_vs_not_uefa_less_valuable['Team.1.Name'] == team) & (team_matches_vs_not_uefa_less_valuable['Score.Difference'] > 0)) |
        ((team_matches_vs_not_uefa_less_valuable['Team.2.Name'] == team) & (team_matches_vs_not_uefa_less_valuable['Score.Difference'] < 0))
    ]

    number_matches_vs_not_uefa_more_valuable = len(team_matches_vs_not_uefa_more_valuable)
    number_matches_vs_not_uefa_less_valuable = len(team_matches_vs_not_uefa_less_valuable)
    number_of_wins_vs_not_uefa_more_valuable = len(wins_vs_not_uefa_more_valuable)
    number_of_wins_vs_not_uefa_less_valuable = len(wins_vs_not_uefa_less_valuable)

    #PLAYING UEFA
    team_matches_vs_uefa_more_valuable = team_matches_vs_uefa[
        ((team_matches_vs_uefa['Team.1.Name'] == team) & (team_matches_vs_uefa['Team.1.Total.Market.Value'] > team_matches_vs_uefa['Team.2.Total.Market.Value'])) |
        ((team_matches_vs_uefa['Team.2.Name'] == team) & (team_matches_vs_uefa['Team.1.Total.Market.Value'] < team_matches_vs_uefa['Team.2.Total.Market.Value']))
    ]
    team_matches_vs_uefa_less_valuable = team_matches_vs_uefa[
        ((team_matches_vs_uefa['Team.1.Name'] == team) & (team_matches_vs_uefa['Team.1.Total.Market.Value'] < team_matches_vs_uefa['Team.2.Total.Market.Value'])) |
        ((team_matches_vs_uefa['Team.2.Name'] == team) & (team_matches_vs_uefa['Team.1.Total.Market.Value'] > team_matches_vs_uefa['Team.2.Total.Market.Value']))
    ]

    wins_vs_uefa_more_valuable = team_matches_vs_uefa_more_valuable[
        ((team_matches_vs_uefa_more_valuable['Team.1.Name'] == team) & (team_matches_vs_uefa_more_valuable['Score.Difference'] > 0)) |
        ((team_matches_vs_uefa_more_valuable['Team.2.Name'] == team) & (team_matches_vs_uefa_more_valuable['Score.Difference'] < 0))
    ]

    wins_vs_uefa_less_valuable = team_matches_vs_uefa_less_valuable[
        ((team_matches_vs_uefa_less_valuable['Team.1.Name'] == team) & (team_matches_vs_uefa_less_valuable['Score.Difference'] > 0)) |
        ((team_matches_vs_uefa_less_valuable['Team.2.Name'] == team) & (team_matches_vs_uefa_less_valuable['Score.Difference'] < 0))
    ]

    number_of_matches_vs_uefa_more_valuable = len(team_matches_vs_uefa_more_valuable)
    number_of_matches_vs_uefa_less_valuable = len(team_matches_vs_uefa_less_valuable)
    number_of_wins_vs_uefa_more_valuable = len(wins_vs_uefa_more_valuable)
    number_of_wins_vs_uefa_less_valuable = len(wins_vs_uefa_less_valuable)

    #PERCENTAGES
    if number_of_matches_vs_uefa_more_valuable != 0:
        win_pct_more_val_uefa = number_of_wins_vs_uefa_more_valuable / number_of_matches_vs_uefa_more_valuable
    else:
        win_pct_more_val_uefa = 0
    
    if number_of_matches_vs_uefa_less_valuable != 0:
        win_pct_less_val_uefa = number_of_wins_vs_uefa_less_valuable / number_of_matches_vs_uefa_less_valuable
    else:
        win_pct_less_val_uefa = 0
    
    if number_matches_vs_not_uefa_less_valuable != 0:
        win_pct_less_val_non_uefa = number_of_wins_vs_not_uefa_less_valuable / number_matches_vs_not_uefa_less_valuable
    else:
        win_pct_less_val_non_uefa = 0
    
    if number_matches_vs_not_uefa_more_valuable != 0:
        win_pct_more_val_non_uefa = number_of_wins_vs_not_uefa_more_valuable / number_matches_vs_not_uefa_more_valuable
    else:
        win_pct_more_val_non_uefa = 0
    
    win_stats = {
        'Team Name': team,
        'Number of Matches vs UEFA as the More Valuable team': number_of_matches_vs_uefa_more_valuable,
        'Number of Wins vs UEFA as the More Valuable team': number_of_wins_vs_uefa_more_valuable,
        'Win PCT vs UEFA as the More Valuable team': win_pct_more_val_uefa,

        'Number of Matches vs Non-UEFA as the More Valuable team': number_matches_vs_not_uefa_more_valuable,
        'Number of Wins vs Non-UEFA as the More Valuable team': number_of_wins_vs_not_uefa_more_valuable,
        'Win PCT vs Non-UEFA as the More Valuable team': win_pct_more_val_non_uefa,

        'Number of Matches vs UEFA as the Less Valuable team': number_of_matches_vs_uefa_less_valuable,
        'Number of Wins vs UEFA as the Less Valuable team': number_of_wins_vs_uefa_less_valuable,
        'Win PCT vs UEFA as the Less Valuable team': win_pct_less_val_uefa,

        'Number of Matches vs Non-UEFA as the Less Valuable team': number_matches_vs_not_uefa_less_valuable,
        'Number of Wins vs Non-UEFA as the Less Valuable team': number_of_wins_vs_not_uefa_less_valuable,
        'Win PCT vs Non-UEFA as the Less Valuable team': win_pct_less_val_non_uefa
    }
    win_stats_df = pd.DataFrame([win_stats])
    # Append to DataFrame
    win_pct_stats = pd.concat([win_pct_stats, win_stats_df], ignore_index=True)
win_pct_stats

,Team Name,Number of Matches vs UEFA as the More Valuable team,Number of Wins vs UEFA as the More Valuable team,Win PCT vs UEFA as the More Valuable team,Number of Matches vs Non-UEFA as the More Valuable team,Number of Wins vs Non-UEFA as the More Valuable team,Win PCT vs Non-UEFA as the More Valuable team,Number of Matches vs UEFA as the Less Valuable team,Number of Wins vs UEFA as the Less Valuable team,Win PCT vs UEFA as the Less Valuable team,Number of Matches vs Non-UEFA as the Less Valuable team,Number of Wins vs Non-UEFA as the Less Valuable team,Win PCT vs Non-UEFA as the Less Valuable team
0,Belgium,79,63,0.797468,9,8,0.888889,8,4,0.500000,1,0,0.000000
1,Bosnia,50,30,0.600000,2,1,0.500000,24,2,0.083333,1,0,0.000000
2,Croatia,62,40,0.645161,6,5,0.833333,24,7,0.291667,5,2,0.400000
3,Denmark,52,35,0.673077,6,2,0.333333,29,10,0.344828,2,0,0.000000
4,England,90,58,0.644444,8,5,0.625000,0,0,0.000000,0,0,0.000000
5,France,74,50,0.675676,14,9,0.642857,5,2,0.400000,0,0,0.000000
6,Germany,70,53,0.757143,15,8,0.533333,9,1,0.111111,2,0,0.000000
7,Greece,47,28,0.595745,2,0,0.000000,27,7,0.259259,2,1,0.500000
8,Iceland,25,17,0.680000,0,0,0.000000,55,14,0.254545,2,0,0.000000
9,Italy,72,51,0.708333,2,0,0.000000,13,5,0.384615,0,0,0.000000
